In [1]:
import pandas as pd
from tqdm import tqdm
from natsort import natsorted
import os, json, open_clip, torch
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

import os
import json
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from natsort import natsorted
import pandas as pd
from tqdm import tqdm
import open_clip
from datetime import datetime

In [2]:
# 0. Settings
device = 'cuda:0'
model_name = 'ViT-L-14'
pretrained = 'openai'
root = './'
dataset_name = 'Scene'

# 1. Load CLIP model
model, _, preprocess = open_clip.create_model_and_transforms(model_name, pretrained=pretrained)
tokenizer = open_clip.get_tokenizer(model_name)

# 2. Load test dataset with data augmentation
ds = ImageFolder(os.path.join(root, dataset_name), transform=preprocess)
ds.samples = natsorted(ds.samples)
dl = DataLoader(ds, shuffle=False, batch_size=32, num_workers=2)

# 3. Load class name list
with open(os.path.join(root, 'classes.json'), 'r') as j:
     class_names = json.loads(j.read())

# Enhanced text prompts with detailed descriptions
class_descriptions = {
    "Buildings": "A photo of buildings, which can include skyscrapers, residential houses, or office complexes. These structures are designed for various human activities and can be found in urban or suburban areas.",
    "Forests": "A photo of a forest, showcasing a dense collection of trees, plants, and wildlife. Forests can vary from tropical rainforests to temperate woodlands, providing habitats for numerous species.",
    "Glacier": "A photo of a glacier, featuring large masses of ice and snow, often found in polar regions or high mountains. Glaciers slowly move over land, shaping the landscape through erosion.",
    "Mountains": "A photo of mountains, displaying tall, rugged peaks, rocky terrain, and often snow-capped summits. Mountains are formed through tectonic forces and are a prominent feature of the Earth's surface.",
    "Sea": "A photo of the sea, depicting vast expanses of saltwater, waves, and sometimes coastal features like beaches or cliffs. The sea is a critical component of the Earth's hydrosphere and supports a diverse range of marine life.",
    "Street": "A photo of a street, including elements like roads, buildings, cars, and people. Streets are essential parts of urban and suburban infrastructure, facilitating transportation and social interaction."
}

# Detailed prompts
prompts = [class_descriptions[class_name] for class_name in class_names]

# 4. Perform zero-shot classification
zero_shot_top1 = 0
submission = dict({'id_idx': list(range(8100)), 'label': []})
with torch.no_grad(), torch.cuda.amp.autocast():
    text = tokenizer(prompts)
    text_features = model.encode_text(text)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    model = model.to(device)
    for x, y in tqdm(dl):
        x = x.cuda(device)
        image_features = model.encode_image(x).to('cpu').float()
        image_features /= image_features.norm(dim=-1, keepdim=True)
        zero_shot_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
        zero_shot_pred = zero_shot_probs.max(dim=-1)[1].tolist()
        submission['label'] += zero_shot_pred

# 5. Save prediction as submission.scv file.
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
file_name = f'submission_{current_time}.csv'
pd.DataFrame(submission).to_csv(os.path.join(root+"submissions/", file_name), index=False)

100%|████████████████████████████████████████████████| 254/254 [00:51<00:00,  4.95it/s]
